In [26]:
import numpy as np
import re
from functools import reduce
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import helpers

In [ ]:
start_time = time.time()

In [3]:
preproc.do_preprocessing('../twitter-datasets/train_pos_full.txt')
preproc.do_preprocessing('../twitter-datasets/train_neg_full.txt')

In [9]:
with open('../twitter-datasets/train_pos_full_processed.txt') as pos_in, open(
                            '../twitter-datasets/train_neg_full_processed.txt') as neg_in:
    pos_lines = pos_in.readlines()
    neg_lines = neg_in.readlines()
    pos_in.close()
    neg_in.close()
lines = pos_lines + neg_lines
lines = [line.replace('\n', '') for line in lines]

In [10]:
print(len(lines))
lines[:5]

2500000


['i duno justin read my mention or not . only justin and god knows about that , but i hope you wil folow me #believe',
 'because your logic is so dumb , i wil not even crop out your name or your photo . tsk .',
 '" just put casper in a box ! " loved the batle ! #crakbitch',
 'thanks sir > > do not trip lil mama . just kep doin ya thang !',
 'visiting my brother tmr is the bestest birthday gift ever ! ! !']

In [11]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lines)

y = np.zeros(shape=(len(lines)))
y[:len(pos_lines)] = 1
y[len(pos_lines):] = -1

In [13]:
clf = LogisticRegression().fit(X, y)
# kf = KFold(n_splits=4, shuffle=True, random_state=0)
# scores = cross_val_score(clf, X, y, cv=kf)
# print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
preproc.do_preprocessing('../twitter-datasets/test_data.txt')

In [21]:
with open('../twitter-datasets/test_data_processed.txt') as test_in:
    test_lines = test_in.readlines()
test_lines = [line.replace('\n', '') for line in test_lines]

In [22]:
print(len(test_lines))
print(test_lines[:5])

10000
['sea do pro sea scoter ( sports with the portable sea-do seascotersave air , stay longer in the water and .', 'shucks wel i work al wek so now i ca not come cher you on ! oh and put those bateries in your calculator ! ! !', 'i canot stay away from bug thats my baby', 'no madam ! ! ! lol im perfectly fine and not contagious anymore lmao', 'whenever i fal aslep watching the tv , i always wake up with a headache']


In [27]:
X_test = vectorizer.transform(test_lines)

y_pred = clf.predict(X_test)

helpers.create_submission_csv(y_pred)

In [28]:
print(np.unique(y_pred))

[-1.  1.]


In [29]:
elapsed_time = divmod(round((time.time() - start_time)), 60)
print('------\nElapsed time: {m} min {s} sec\n'.format(m=elapsed_time[0], s=elapsed_time[1]))

------
Elapsed time: 27 min 31 sec

